In [ ]:
import math, random

vocab_size, embed_dim = 10, 4
hidden_1, hidden_2 = 8, 4
lr, l1_param, l2_param = 0.05, 0.001, 0.01
margin, eps, mucoef, vecoef = 2.0, 1e-8, 0.9, 0.999

embedding_table = [[random.uniform(-0.1, 0.1) for _ in range(embed_dim)] for _ in range(vocab_size)]
neurons = [[random.uniform(-0.1, 0.1) for _ in range(embed_dim)] for _ in range(hidden_1)]
layer_2 = [[random.uniform(-0.1, 0.1) for _ in range(hidden_1)] for _ in range(hidden_2)]
weights = [[random.uniform(-0.1, 0.1) for _ in range(hidden_2)] for _ in range(2)]

n_bias, l2_bias, w_bias = [0.0]*hidden_1, [0.0]*hidden_2, [0.0]*2

mu_e, ve_e = [[0.0]*embed_dim for _ in range(vocab_size)], [[0.0]*embed_dim for _ in range(vocab_size)]
mu_n, ve_n = [[0.0]*embed_dim for _ in range(hidden_1)], [[0.0]*embed_dim for _ in range(hidden_1)]
mu_l2, ve_l2 = [[0.0]*hidden_1 for _ in range(hidden_2)], [[0.0]*hidden_1 for _ in range(hidden_2)]
mu_w, ve_w = [[0.0]*hidden_2 for _ in range(2)], [[0.0]*hidden_2 for _ in range(2)]

mu_nb, ve_nb = [0.0]*hidden_1, [0.0]*hidden_1
mu_l2b, ve_l2b = [0.0]*hidden_2, [0.0]*hidden_2
mu_wb, ve_wb = [0.0]*2, [0.0]*2
dataset, target = [0, 1, 2, 3, 4], [[1, 0], [0, 1], [1, 0], [0, 1], [1, 0]]

batch_size, batch_count = 3, 0
acc_w = [[0.0]*hidden_2 for _ in range(2)]; acc_wb = [0.0]*2
acc_l2 = [[0.0]*hidden_1 for _ in range(hidden_2)]; acc_l2b = [0.0]*hidden_2
acc_n = [[0.0]*embed_dim for _ in range(hidden_1)]; acc_nb = [0.0]*hidden_1
acc_e = [[0.0]*embed_dim for _ in range(vocab_size)]

dropout = 0.2
t = 0

for epoch in range(500):
    data_indices = list(range(len(dataset)))
    random.shuffle(data_indices)
    for batch_idx, i in enumerate(data_indices):
        batch_count += 1
        emb_idx = dataset[i]
        vectors = embedding_table[emb_idx]

        n_logits_pre = [max(0, sum(v * w for v, w in zip(vectors, row)) + n_bias[idx]) for idx, row in enumerate(neurons)]
        n_mask = [1 if random.random() > dropout else 0 for _ in range(hidden_1)]
        n_logits = [(val * m) / (1 - dropout) for val, m in zip(n_logits_pre, n_mask)]
        l2_logits_pre = [max(0, sum(v * w for v, w in zip(n_logits, row)) + l2_bias[idx]) for idx, row in enumerate(layer_2)]
        l2_mask = [1 if random.random() > dropout else 0 for _ in range(hidden_2)]
        l2_logits = [(val * m) / (1 - dropout) for val, m in zip(l2_logits_pre, l2_mask)]
        logits = [sum(v * w for v, w in zip(l2_logits, row)) + w_bias[idx] for idx, row in enumerate(weights)]
        train_logits = list(logits)
        correct_idx = target[i].index(1)
        train_logits[correct_idx] /= margin
        probs = [math.exp(l) / sum(math.exp(x) for x in train_logits) for l in train_logits]

        w_loss = [probs[j] - target[i][j] for j in range(2)]
        l2_loss = [sum(w_loss[k] * weights[k][j] for k in range(2)) * (1 if l2_logits_pre[j] > 0 else 0) * l2_mask[j] for j in range(hidden_2)]
        n_loss = [sum(l2_loss[k] * layer_2[k][j] for k in range(hidden_2)) * (1 if n_logits_pre[j] > 0 else 0) * n_mask[j] for j in range(hidden_1)]
        e_loss = [sum(n_loss[j] * neurons[j][k] for j in range(hidden_1)) for k in range(embed_dim)]
        for j in range(2):
            for k in range(hidden_2): acc_w[j][k] += w_loss[j] * l2_logits[k]
            acc_wb[j] += w_loss[j]
        for j in range(hidden_2):
            for k in range(hidden_1): acc_l2[j][k] += l2_loss[j] * n_logits[k]
            acc_l2b[j] += l2_loss[j]
        for j in range(hidden_1):
            for k in range(embed_dim): acc_n[j][k] += n_loss[j] * vectors[k]
            acc_nb[j] += n_loss[j]
        for k in range(embed_dim): acc_e[emb_idx][k] += e_loss[k]

        if batch_count == batch_size or (batch_idx + 1) == len(dataset):
            t += 1
            n_samples = batch_count 
            
            for j in range(2):
                for k in range(hidden_2):
                    grad = acc_w[j][k] / n_samples
                    reg = l2_param * weights[j][k] + l1_param * (1 if weights[j][k] > 0 else -1)
                    l2grad = grad + reg
                    mu_w[j][k] = mucoef * mu_w[j][k] + (1 - mucoef) * l2grad
                    ve_w[j][k] = vecoef * ve_w[j][k] + (1 - vecoef) * (l2grad**2)
                    weights[j][k] -= lr * (mu_w[j][k]/(1-mucoef**t)) / (math.sqrt(ve_w[j][k]/(1-vecoef**t)) + eps)
                
                bias_grad = acc_wb[j] / n_samples
                mu_wb[j] = mucoef * mu_wb[j] + (1 - mucoef) * bias_grad
                ve_wb[j] = vecoef * ve_wb[j] + (1 - vecoef) * (bias_grad**2)
                w_bias[j] -= lr * (mu_wb[j]/(1-mucoef**t)) / (math.sqrt(ve_wb[j]/(1-vecoef**t)) + eps)

            for j in range(hidden_2):
                for k in range(hidden_1):
                    grad = acc_l2[j][k] / n_samples 
                    l2grad = grad + (l2_param * layer_2[j][k])
                    mu_l2[j][k] = mucoef * mu_l2[j][k] + (1 - mucoef) * l2grad
                    ve_l2[j][k] = vecoef * ve_l2[j][k] + (1 - vecoef) * (l2grad**2)
                    layer_2[j][k] -= lr * (mu_l2[j][k]/(1-mucoef**t)) / (math.sqrt(ve_l2[j][k]/(1-vecoef**t)) + eps)
                
                bias_grad = acc_l2b[j] / n_samples
                mu_l2b[j] = mucoef * mu_l2b[j] + (1 - mucoef) * bias_grad
                ve_l2b[j] = vecoef * ve_l2b[j] + (1 - vecoef) * (bias_grad**2)
                l2_bias[j] -= lr * (mu_l2b[j]/(1-mucoef**t)) / (math.sqrt(ve_l2b[j]/(1-vecoef**t)) + eps)

            for j in range(hidden_1):
                for k in range(embed_dim):
                    grad = acc_n[j][k] / n_samples
                    l2grad = grad + (l2_param * neurons[j][k])
                    mu_n[j][k] = mucoef * mu_n[j][k] + (1 - mucoef) * l2grad
                    ve_n[j][k] = vecoef * ve_n[j][k] + (1 - vecoef) * (l2grad**2)
                    neurons[j][k] -= lr * (mu_n[j][k]/(1-mucoef**t)) / (math.sqrt(ve_n[j][k]/(1-vecoef**t)) + eps)
                
                bias_grad = acc_nb[j] / n_samples
                mu_nb[j] = mucoef * mu_nb[j] + (1 - mucoef) * bias_grad
                ve_nb[j] = vecoef * ve_nb[j] + (1 - vecoef) * (bias_grad**2)
                n_bias[j] -= lr * (mu_nb[j]/(1-mucoef**t)) / (math.sqrt(ve_nb[j]/(1-vecoef**t)) + eps)

            for idx in range(vocab_size):
                for k in range(embed_dim):
                    if any(acc_e[idx]):
                        grad = acc_e[idx][k] / n_samples
                        mu_e[idx][k] = mucoef * mu_e[idx][k] + (1 - mucoef) * grad
                        ve_e[idx][k] = vecoef * ve_e[idx][k] + (1 - vecoef) * (grad**2)
                        embedding_table[idx][k] -= lr * (mu_e[idx][k]/(1-mucoef**t)) / (math.sqrt(ve_e[idx][k]/(1-vecoef**t)) + eps)
            
            acc_w = [[0.0]*hidden_2 for _ in range(2)]; acc_wb = [0.0]*2
            acc_l2 = [[0.0]*hidden_1 for _ in range(hidden_2)]; acc_l2b = [0.0]*hidden_2
            acc_n = [[0.0]*embed_dim for _ in range(hidden_1)]; acc_nb = [0.0]*hidden_1
            acc_e = [[0.0]*embed_dim for _ in range(vocab_size)]
            batch_count = 0

print("\n--- Testing Results ---")
for i in range(len(dataset)):
    emb_idx = dataset[i]
    vectors = embedding_table[emb_idx]
    n_l = [max(0, sum(v * w for v, w in zip(vectors, row)) + n_bias[idx]) for idx, row in enumerate(neurons)]
    l2_l = [max(0, sum(v * w for v, w in zip(n_l, row)) + l2_bias[idx]) for idx, row in enumerate(layer_2)]
    logits = [sum(v * w for v, w in zip(l2_l, row)) + w_bias[idx] for idx, row in enumerate(weights)]
    exp_logits = [math.exp(l) for l in logits]
    probs = [round(e / sum(exp_logits), 3) for e in exp_logits]
    print(f"Word Index {emb_idx} | Target: {target[i]} | Probs: {probs}")


--- Testing Results ---
Word Index 0 | Target: [1, 0] | Probs: [0.637, 0.363]
Word Index 1 | Target: [0, 1] | Probs: [0.637, 0.363]
Word Index 2 | Target: [1, 0] | Probs: [0.637, 0.363]
Word Index 3 | Target: [0, 1] | Probs: [0.637, 0.363]
Word Index 4 | Target: [1, 0] | Probs: [0.637, 0.363]


In [ ]:
old_games = [
    [0.1, 0.8, 0.1],
    [0.9, 0.9, 0.9], 
    [0.8, 0.2, 0.5],
]
results = [0, 0, 1]

def knn_predict(new_data, dataset, targets, k=3):
    distances = []
    for i in range(len(dataset)):
        dist = sum((new_data[j] - dataset[i][j])**2 for j in range(len(new_data)))**0.5
        distances.append((dist, targets[i]))
    distances.sort(key=lambda x: x[0])
    nearest = [d[1] for d in distances[:k]]
    return "Хит" if max(set(nearest), key=nearest.count) == 0 else "Провал"

test_game = [0.15, 0.85, 0.12]
print(f"Вердикт KNN: {knn_predict(test_game, old_games, results, k=1)}")

In [ ]:
def tree_predict(game):
    if game[1] > 0.8:
        if game[0] < 0.3: return "Инди-хит"
        else: return "Блокбастер"
    else: return "Провал"

new_game = [0.1, 0.9, 0.5]
print(f"Вердикт дерева: {tree_predict(new_game)}")

In [ ]:
import random
dataset = [[0.1, 0.8, 0.1], [0.9, 0.9, 0.9], [0.8, 0.2, 0.5]]
targets = [1, 1, -1] 
weights = [random.uniform(-0.1, 0.1) for _ in range(3)]
bias = 0.0
lr = 0.01 * 2
C = 1.0
epochs = 1000
l1_param = 0.005
l2_param = 0.01

for epoch in range(epochs):
    for i, x in enumerate(dataset):
        condition = targets[i] * (sum(x[j] * weights[j] for j in range(3)) + bias)
        if condition >= 1:
            for j in range(3):
                weights[j] -= lr * (l2_param * weights[j] + l1_param * (1 if weights[j] > 0 else -1))
        else:
            for j in range(3):
                weights[j] -= lr * (l2_param * weights[j] - C * x[j] * targets[i])
            bias += lr * C * targets[i]

test_game = [0.15, 0.85, 0.12]
result = sum(test_game[j] * weights[j] for j in range(3)) + bias
def sigmoid(z):
    return 1 / (1 + math.exp(-z))
prob_hit = sigmoid(result)
print(f"SVM вердикт: {'Хит' if result > 0 else 'Провал'}")
print(f"Счет (Score): {result:.2f}")
print(f"Уверенность (Вероятность Хима): {prob_hit:.2%}")